In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
# Jリーグ公式データサイト
BASE_URL = "https://data.j-league.or.jp/SFIX06/searchStaffListInfoByFirstAlphabet?staffNameFirstAlphabet="
ALPHABETS = ["あ","い","う","え","お",
              "か","き","く","け","こ",
              "さ","し","す","せ","そ",
              "た","ち","つ","て","と",
              "な","に","ぬ","ね","の",
              "は","ひ","ふ","へ","ほ",
              "ま","み","む","め","も",
              "や","ゆ","よ",
              "ら","り","る","れ","ろ",
              "わ"]

BASE_SITE = "https://data.j-league.or.jp"
results = []

for a in ALPHABETS:
    url = BASE_URL + a
    print(f"Fetching {a} ...")
    res = requests.get(url)
    res.encoding = res.apparent_encoding  # 日本語文字化け防止
    soup = BeautifulSoup(res.text, "html.parser")

    # <td class="bl-non"> を全取得
    for td in soup.select("td.bl-non"):
        a_tag = td.find("a")
        if a_tag and "href" in a_tag.attrs:
            name = a_tag.text.strip()
            link = BASE_SITE + a_tag["href"]
            results.append({"name": name, "url": link})
    
    time.sleep(1)  # アクセスマナー

# pandasでまとめて保存
df = pd.DataFrame(results)
df.to_csv("jleague_staff_links.csv", index=False, encoding="utf-8-sig")

print(f"取得完了: {len(df)}件")


Fetching あ ...
Fetching い ...
Fetching う ...
Fetching え ...
Fetching お ...
Fetching か ...
Fetching き ...
Fetching く ...
Fetching け ...
Fetching こ ...
Fetching さ ...
Fetching し ...
Fetching す ...
Fetching せ ...
Fetching そ ...
Fetching た ...
Fetching ち ...
Fetching つ ...
Fetching て ...
Fetching と ...
Fetching な ...
Fetching に ...
Fetching ぬ ...
Fetching ね ...
Fetching の ...
Fetching は ...
Fetching ひ ...
Fetching ふ ...
Fetching へ ...
Fetching ほ ...
Fetching ま ...
Fetching み ...
Fetching む ...
Fetching め ...
Fetching も ...
Fetching や ...
Fetching ゆ ...
Fetching よ ...
Fetching ら ...
Fetching り ...
Fetching る ...
Fetching れ ...
Fetching ろ ...
Fetching わ ...
取得完了: 445件


In [3]:
# === 入力：監督一覧 ===
links_df = pd.read_csv("jleague_staff_links.csv")

records = []

for i, row in links_df.iterrows():
    url = row["url"]
    staff_name = row["name"]
    print(f"{i+1}/{len(links_df)}: {staff_name}")

    try:
        res = requests.get(url)
        res.encoding = res.apparent_encoding
        soup = BeautifulSoup(res.text, "html.parser")

        # 各行（年度別の成績）を抽出
        for tr in soup.select("tbody tr"):
            tds = tr.find_all("td")
            if len(tds) < 9:
                continue

            year = tds[0].get_text(strip=True)  # bl-non（年度）
            team = tds[2].get_text(strip=True)
            league = tds[3].get_text(strip=True)
            games = tds[4].get_text(strip=True)
            wins = tds[5].get_text(strip=True)
            draws = tds[6].get_text(strip=True)
            losses = tds[7].get_text(strip=True)
            result = tds[8].get_text(strip=True)

            records.append({
                "監督名": staff_name,
                "年度": year,
                "チーム": team,
                "大会": league,
                "試合数": games,
                "勝": wins,
                "分": draws,
                "負": losses,
                "成績": result,
                "URL": url
            })

    except Exception as e:
        print(f"Error at {url}: {e}")
    
    time.sleep(1)

# === CSVに出力 ===
df = pd.DataFrame(records)
df.to_csv("jleague_manager_results.csv", index=False, encoding="utf-8-sig")

print(f"完了: {len(df)} 行を保存しました。")

1/445: 青野　慎也
2/445: 秋田　豊
3/445: 秋葉　忠宏
4/445: 浅野　哲也
5/445: アジウソン
6/445: アスカルゴルタ
7/445: 東　泰
8/445: 足達　勇輔
9/445: 安達　亮
10/445: アフシン　ゴトビ
11/445: 天野　賢一
12/445: 雨野　裕介
13/445: アマル　オシム
14/445: 有馬　賢二
15/445: アルベル
16/445: アルベルト
17/445: アレックス　ミラー
18/445: アレッシャンドレ　ガーロ
19/445: アンジェ　ポステコグルー
20/445: アントニーニョ
21/445: アントネッティ
22/445: 安間　貴義
23/445: ア・デモス
24/445: アーサー　パパス
25/445: アーリ　スカンズ
26/445: 池田　伸康
27/445: 池谷　友良
28/445: 石井　肇
29/445: 石井　正忠
30/445: 石﨑　信弘
31/445: 石丸　清隆
32/445: 和泉　茂徳
33/445: 伊藤　彰
34/445: 井原　正巳
35/445: イバンチェビッチ
36/445: 今井　敏明
37/445: 今井　雅隆
38/445: 今矢　直城
39/445: 入江　徹
40/445: 岩瀬　健
41/445: 岩政　大樹
42/445: 植木　繁晴
43/445: 上田　栄治
44/445: 上野　展裕
45/445: 上野　優作
46/445: 上野山　信行
47/445: 上村　健一
48/445: 浮氣　哲郎
49/445: 浮嶋　敏
50/445: 内田　一夫
51/445: 内山　篤
52/445: ヴァレリー
53/445: 江尻　篤彦
54/445: エスピノーザ
55/445: 衛藤　元
56/445: エドゥ
57/445: エメルソン　レオン
58/445: エリク　モンバエルツ
59/445: エルスナー
60/445: 大石　篤人
61/445: 大岩　剛
62/445: 大榎　克己
63/445: 大木　武
64/445: 大熊　清
65/445: 大熊　裕司
66/445: 大島　秀夫
67/445: 大島　康明
68/445: 大嶽　直人
69/445: 大塚　真司
70/445: 大槻　